# Demo of PSyKI

This is a brief demonstration of PSyKI.
All main symbolic knowledge injection workflow steps will be addressed.


Some utility imports.

In [1]:
from logic.datalog.grammar.adapters.antlr4 import get_formula_from_string
from test.resources.rules.poker import FEATURE_MAPPING as POKER_FEATURE_MAPPING
from test.resources.rules.poker import CLASS_MAPPING as POKER_CLASS_MAPPING
from test.utils import get_processed_dataset
from test.resources.rules import get_rules

Loading poker dataset and separation into train and test set.

In [2]:
feature_mapping = POKER_FEATURE_MAPPING
class_mapping = POKER_CLASS_MAPPING
train_x, train_y, test_x, test_y = get_processed_dataset('poker')
train_x

array([[ 1., 10.,  1., ..., 12.,  1.,  1.],
       [ 2., 11.,  2., ..., 12.,  2.,  1.],
       [ 3., 12.,  3., ..., 10.,  3.,  1.],
       ...,
       [ 2.,  1.,  2., ...,  1.,  4., 13.],
       [ 2., 12.,  4., ..., 12.,  4.,  9.],
       [ 1.,  7.,  3., ...,  8.,  3.,  7.]])

Import Datalog poker rules and parsing into Formula data structure.

In [3]:
poker_rules = get_rules('poker')
formulae = [get_formula_from_string(rule) for rule in poker_rules]
str(formulae[0])

'pair(S1,R1,S2,R2,S3,R3,S4,R4,S5,R5)←((R1)=(R2))'

## Injection via structuring using NetworkComposer

In [4]:
from tensorflow.keras import Input
from psyki.ski.injectors import NetworkComposer
from tensorflow.keras.models import Model
from test.utils import get_mlp
from tensorflow.python.framework.random_seed import set_random_seed

set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = NetworkComposer(model, feature_mapping, layer=len(model.layers)-2)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
lambda_20 (Lambda)              (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
lambda_21 (Lambda)              (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
lambda_22 (Lambda)              (None, 1)            0           Input[0][0]                      
____________________________________________________________________________________________

### Training

In [5]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=20)

2022-06-07 17:08:53.463727: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-07 17:08:53.463865: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20
782/782 [==============================] - 4s 1ms/step - loss: 0.7106 - accuracy: 0.8305
Epoch 2/20
782/782 [==============================] - 1s 1ms/step - loss: 0.3499 - accuracy: 0.9297
Epoch 3/20
782/782 [==============================] - 1s 1ms/step - loss: 0.2089 - accuracy: 0.9543
Epoch 4/20
782/782 [==============================] - 1s 1ms/step - loss: 0.1353 - accuracy: 0.9811
Epoch 5/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0913 - accuracy: 0.9921
Epoch 6/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0636 - accuracy: 0.9953
Epoch 7/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0447 - accuracy: 0.9974
Epoch 8/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0320 - accuracy: 0.9988
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0232 - accuracy: 0.9994
Epoch 10/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0171 - accuracy: 0.9996

### Evaluation
Loss and accuracy

In [6]:
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 30s 924us/step - loss: 0.0022 - accuracy: 0.9997
Loss: 0.0021584308706223965
Accuracy: 0.9997450113296509


In [7]:
from test.utils import get_class_accuracy

accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 1.0
Accuracy of class pair: 1.0
Accuracy of class two: 1.0
Accuracy of class three: 1.0
Accuracy of class straight: 0.9997425997425997
Accuracy of class flush: 1.0
Accuracy of class full: 0.9676966292134831
Accuracy of class four: 0.14782608695652175
Accuracy of class straight_flush: 0.16666666666666666
Accuracy of class royal_flush: 0.3333333333333333


Macro F1-measure

In [8]:
from test.utils import get_f1

f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.8021688605291398


## Injection via constraining using LambdaLayer

In [20]:
from ski.injectors import LambdaLayer

str(formulae[0])
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = LambdaLayer(model, class_mapping, feature_mapping, 1.5)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "constrained_model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_826 (Dense)               (None, 64)           704         Input[0][0]                      
__________________________________________________________________________________________________
dense_827 (Dense)               (None, 64)           4160        dense_826[0][0]                  
__________________________________________________________________________________________________
dense_828 (Dense)               (None, 10)           650         dense_827[0][0]                  
________________________________________________________________________________

### Training

In [ ]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 14s 6ms/step - loss: 1.3367 - accuracy: 0.2515
Epoch 2/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2525 - accuracy: 0.3006
Epoch 3/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2330 - accuracy: 0.3444
Epoch 4/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2209 - accuracy: 0.3772
Epoch 5/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2096 - accuracy: 0.3839
Epoch 6/100
782/782 [==============================] - 5s 6ms/step - loss: 1.2007 - accuracy: 0.3968
Epoch 7/100
782/782 [==============================] - 5s 6ms/step - loss: 1.1895 - accuracy: 0.4012
Epoch 8/100
782/782 [==============================] - 5s 6ms/step - loss: 1.1796 - accuracy: 0.4109
Epoch 9/100
782/782 [==============================] - 5s 6ms/step - loss: 1.1678 - accuracy: 0.4160
Epoch 10/100
782/782 [==============================] - 5s 6ms/step - loss: 1.1558 - accur

### Remove constraints and evaluation
Loss and Accuracy

In [19]:
#predictor = injector.remove()
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 6s 205us/step - loss: 7.9064 - accuracy: 0.0022
Loss: 7.906429767608643
Accuracy: 0.0022350000217556953


In [12]:
accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 0.0
Accuracy of class pair: 0.005278131494113582
Accuracy of class two: 2.0998698080718995e-05
Accuracy of class three: 0.00014203872922683586
Accuracy of class straight: 0.0
Accuracy of class flush: 0.0
Accuracy of class full: 0.0
Accuracy of class four: 0.004347826086956522
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [13]:
f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.0005077163705341896


## The same network without knowledge injection

In [14]:
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
dense_823 (Dense)            (None, 64)                704       
_________________________________________________________________
dense_824 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_825 (Dense)            (None, 10)                650       
Total params: 5,514
Trainable params: 5,514
Non-trainable params: 0
_________________________________________________________________


### Training

In [15]:
model.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 0s 376us/step - loss: 1.0413 - accuracy: 0.4912
Epoch 2/100
782/782 [==============================] - 0s 365us/step - loss: 0.9710 - accuracy: 0.5263
Epoch 3/100
782/782 [==============================] - 0s 369us/step - loss: 0.9570 - accuracy: 0.5398
Epoch 4/100
782/782 [==============================] - 0s 383us/step - loss: 0.9496 - accuracy: 0.5455
Epoch 5/100
782/782 [==============================] - 0s 374us/step - loss: 0.9406 - accuracy: 0.5541
Epoch 6/100
782/782 [==============================] - 0s 382us/step - loss: 0.9340 - accuracy: 0.5612
Epoch 7/100
782/782 [==============================] - 0s 368us/step - loss: 0.9261 - accuracy: 0.5595
Epoch 8/100
782/782 [==============================] - 0s 374us/step - loss: 0.9191 - accuracy: 0.5701
Epoch 9/100
782/782 [==============================] - 0s 370us/step - loss: 0.9123 - accuracy: 0.5746
Epoch 10/100
782/782 [==============================] - 0s 369us/step - l

### Evaluation
Loss and Accuracy

In [16]:
loss, accuracy = model.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 6s 204us/step - loss: 0.1261 - accuracy: 0.9664
Loss: 0.1261490285396576
Accuracy: 0.9664030075073242


In [17]:
accuracies, weights = get_class_accuracy(model, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' +str(accuracies[index]))

Accuracy of class nothing: 0.9944434357723025
Accuracy of class pair: 0.966158419684827
Accuracy of class two: 0.8478224350090294
Accuracy of class three: 0.871028833862033
Accuracy of class straight: 0.15933075933075933
Accuracy of class flush: 0.011523046092184368
Accuracy of class full: 0.2478932584269663
Accuracy of class four: 0.05217391304347826
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [18]:
f1 = get_f1(model, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.4355657347081636
